In [20]:
import pandas as pd
from rich import print as rprint
from rich.text import Text
import string
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import re

In [12]:
all_maroon = pd.read_csv("all_issues.csv", index_col = [0])

In [13]:
all_maroon

,year,link,txt,pdf,tag,day,month,year_check
0,1902,https://campub.lib.uchicago.edu/pdf/?docId=mvo...,txt/1902_ocr/ocr_1902_1001.txt,web_scraping/pdfs/1902/1902_1001.pdf,1902_1001,1,10,1902
1,1902,https://campub.lib.uchicago.edu/pdf/?docId=mvo...,txt/1902_ocr/ocr_1902_1002.txt,web_scraping/pdfs/1902/1902_1002.pdf,1902_1002,2,10,1902
2,1902,https://campub.lib.uchicago.edu/pdf/?docId=mvo...,txt/1902_ocr/ocr_1902_1003.txt,web_scraping/pdfs/1902/1902_1003.pdf,1902_1003,3,10,1902
3,1902,https://campub.lib.uchicago.edu/pdf/?docId=mvo...,txt/1902_ocr/ocr_1902_1006.txt,web_scraping/pdfs/1902/1902_1006.pdf,1902_1006,6,10,1902
4,1902,https://campub.lib.uchicago.edu/pdf/?docId=mvo...,txt/1902_ocr/ocr_1902_1007.txt,web_scraping/pdfs/1902/1902_1007.pdf,1902_1007,7,10,1902
...,...,...,...,...,...,...,...,...
7311,1986,https://campub.lib.uchicago.edu/pdf/?docId=mvo...,txt/1986_ocr/ocr_1986_1114.txt,web_scraping/pdfs/1986/1986_1114.pdf,1986_1114,14,11,1986
7312,1986,https://campub.lib.uchicago.edu/pdf/?docId=mvo...,txt/1986_ocr/ocr_1986_1118.txt,web_scraping/pdfs/1986/1986_1118.pdf,1986_1118,18,11,1986
7313,1986,https://campub.lib.uchicago.edu/pdf/?docId=mvo...,txt/1986_ocr/ocr_1986_1121.txt,web_scraping/pdfs/1986/1986_1121.pdf,1986_1121,21,11,1986
7314,1986,https://campub.lib.uchicago.edu/pdf/?docId=mvo...,txt/1986_ocr/ocr_1986_1125.txt,web_scraping/pdfs/1986/1986_1125.pdf,1986_1125,25,11,1986


In [189]:
issue_txt_file = open(all_maroon.txt[5558], "r", encoding='utf-8-sig') #need this encoding to dodge \ufeff
issue_txt_original = issue_txt_file.read()
issue_txt_file.close()

In [186]:
all_maroon.iloc[5558]

year                                                       1954
link          https://campub.lib.uchicago.edu/pdf/?docId=mvo...
txt                              txt/1954_ocr/ocr_1954_1029.txt
pdf                        web_scraping/pdfs/1954/1954_1029.pdf
tag                                                   1954_1029
day                                                          29
month                                                        10
year_check                                                 1954
Name: 5558, dtype: object

In [190]:
issue_txt_original

'SG elects Fortier president;\nby-law amendment adopted\n\nNew officer’s for the 1954-55 Student Government were the principal item of business for\nthe first meeting of the new SG assembly last Tuesday.\n\nAl Fortier (ISL-col) was elected president of SG. Other officers elected were Emil John-\n\nWritedns illegal,\neight ballots void\n\nA major controversy in the re- Asked why he didn’t petition\ncent Student Government elec- and register as an official candi-\n\nson (ISL-col), vice-president; Sabina Wagner (ISL-col), secretary; and Pat Parker (ISL-\nbus), treasurer.\n\nAn amendment to the SG by-laws was adopted which provides that any party receiving\n25 per cent of the total vote in\nan SG election and holding a\nseat in the assembly is entitled\nto a committee-at-large, and thuS\nto a representative on the SG ex-\necutive council. This gives the\nStudent Representative party a\nseat on the council, which it would\nnot otherwise have had.\n\nThe former by-law provided that\nany part

In [194]:
test = "hello\nbob\n\ntompson jones"
#replace only single new lines, but not double new lines. Allows us to keep this whole during cleaning, chop based on double new lines at end
re.sub(r'([^\n])(\n)([^\n])', r"\1 \3", test)

'hello bob\n\ntompson jones'

In [191]:
issue_txt = issue_txt_original
issue_txt = re.sub(r'[^\s]{20,}', " ", issue_txt)
issue_txt = re.sub(r"-\s*\n+", "", issue_txt)
issue_txt.split("\n\n")

['SG elects Fortier president;\nby-law amendment adopted',
 'New officer’s for the 1954-55 Student Government were the principal item of business for\nthe first meeting of the new SG assembly last Tuesday.',
 'Al Fortier (ISL-col) was elected president of SG. Other officers elected were Emil JohnWritedns illegal,\neight ballots void',
 'A major controversy in the re- Asked why he didn’t petition\ncent Student Government elec- and register as an official candison (ISL-col), vice-president; Sabina Wagner (ISL-col), secretary; and Pat Parker (ISLbus), treasurer.',
 'An amendment to the SG by-laws was adopted which provides that any party receiving\n25 per cent of the total vote in\nan SG election and holding a\nseat in the assembly is entitled\nto a committee-at-large, and thuS\nto a representative on the SG executive council. This gives the\nStudent Representative party a\nseat on the council, which it would\nnot otherwise have had.',
 'The former by-law provided that\nany party with 20 

In [205]:
len("""MU 4-0435. John Sutcliffe. Carmen’s used furniture store. Moving and light hauling. 1127 E. 55th St. MU 4-9003. Personal Farewell to all my past ana present instructors and other friends. It has been both a pleasure and a wonderful experience knowing you all. I look forward to the day when we may meet again. Sincerely, Marv Lyonus. CARMEN\'S Used Furniture Store Typewriters and Desks Moving and Light Hauling 1127 E. 55th MU 4-9003 SAVE TIME — SAVE MONEY N & R Bonded Movers — also — New and Used Furniture Mi 3-4196 1510 E. 55th The College LAUNDERETTE 1449 East 57th St. MU 4-9236 BETZ JEWELRY Unusual Jewelry Our Specialty Expert Jewelry and Watch Repair N.S.A. Discount to Students 1523 E. 53rd PL 2-3038 Choose an Expert THOMPSON MOVERS Local and Long Distance Moving CO 4-7600 Bibo, ergo sum Jimmy’s 1172 E. 55th MIDWAY 3-4433 1171 EAST 55th STREET ACASA Book Store Scholarly used books Typewriters bought, sold and repaired 1117 E. 55lli St. Hl 3-9651 new and different stag or drag Dancing and Entertainment Tonight Music by Eddie Fox, His Orchestra and Vocalist from 9:00 P.M. Admission $1.00 plus tax and that\'s all The Elms Hotel, East 53rd Street and Cornell Avenue We stock most blends of Twining Teas. Earl Grey, $1.75 Jasmine, $2.00. In half pound canisters. GET MUCH MORE FLAVOR MUCH LESS NICOTINE I Jisquhr. *WQ It\'s the FILTER that Counts and L*M has the Best! give you the Miracle Tip — the effec* tive filtration you need.. Cet much more flavor — much less nicotine — a light, mild smoke. Yes, — it s the filter that counts and L&M has the best! BUY L&Ms King-size or regular. JUST WHAT THE DOCTOR ORDERED! Why do L&M sales soar higher every day? It’s the one filter cigarette that gives smokers a taste they can enjoy — a filter they can depend on. Now L&M comes king-size, too, at the same low price as regular. In either size —only L&M Filters 1 ^Yers ION'""")

1881

In [199]:
#\s means all whitespace characters (regular spaces, tabs, etc.). Just putting in a space only gets regular spaces

#sometimes boxes around a thing get turned into long chains of letters, usually i, so if there is a stretch of characters surrounded by spaces at least 20 long, remove it
issue_txt = re.sub(r'[^\s]{20,}', " ", issue_txt_original)

#words that are spread across multiple lines, marked with a dash to show continuation
# this isn't perfect. There are lines where the line ends with a dash but the subsequent line is the start of a different paragraph. Like 20-30% of the time. Still worth it to do, but a reason to chop up the newspapers in the future.
issue_txt = re.sub(r"-\s*\n+", "", issue_txt)

#remove repeated stretches of certain kinds of punctuation
# - and — are actually different! N and M dashes?
issue_txt = re.sub(r'[.,_►<>♦❖\^\-\—\s]{5,}', " ", issue_txt)

#possibly remove: ■ * • | «» ♦
issue_txt = issue_txt.replace('■', "")#.replace('*', " ").replace("•", " ").replace("|", " ").replace(" - ", " ")

#remove SINGLE newlines, tabs, etc.
issue_txt = re.sub(r'([^\n])(\n)([^\n])', r"\1 \3", issue_txt)
issue_txt = re.sub(r'\t|\x0c', " ", issue_txt)

#if there is a lowercase followed by a capital with no space between, insert one. Probably two words smushed together
issue_txt = re.sub(r'([a-z])([A-Z])', r'\1 \2', issue_txt)

#remove multiple spaces
issue_txt = re.sub(' +', " ", issue_txt)

#splitting on two or greater new lines
para_lst = re.split(r'\n{2,}', issue_txt)

#should we repalce "\'" with "’"? 
#"^ is our most common non-standard character, but there is no consistency in what it actually is. Sometimes it is for e, other times for a, other times a space. Not sure how to deal wwith it."

[53,
 143,
 124,
 225,
 367,
 256,
 122,
 157,
 280,
 255,
 26,
 17,
 34,
 136,
 22,
 108,
 310,
 335,
 32,
 20,
 59,
 198,
 202,
 231,
 191,
 287,
 259,
 15,
 105,
 125,
 267,
 188,
 17,
 177,
 202,
 312,
 248,
 19,
 13,
 24,
 22,
 30,
 21,
 17,
 20,
 7,
 15,
 25,
 12,
 300,
 208,
 19,
 75,
 36,
 346,
 254,
 29,
 14,
 18,
 17,
 51,
 180,
 399,
 8,
 11,
 8,
 18,
 37,
 21,
 27,
 10,
 14,
 50,
 55,
 10,
 11,
 56,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 15,
 9,
 8,
 8,
 8,
 8,
 34,
 20,
 17,
 83,
 357,
 221,
 350,
 72,
 261,
 83,
 230,
 198,
 18,
 16,
 8,
 15,
 246,
 89,
 17,
 105,
 109,
 15,
 201,
 215,
 228,
 172,
 101,
 14,
 69,
 61,
 214,
 243,
 98,
 15,
 56,
 98,
 197,
 118,
 130,
 107,
 92,
 58,
 42,
 128,
 142,
 163,
 16,
 37,
 45,
 211,
 181,
 118,
 217,
 160,
 10,
 11,
 22,
 27,
 25,
 27,
 28,
 8,
 59,
 52,
 29,
 40,
 26,
 28,
 41,
 16,
 13,
 1,
 11,
 278,
 32,
 15,
 16,
 15,
 15,
 23,
 30,
 657,
 20,
 27,
 4,
 8,
 16,
 20,
 17,
 17,
 9,
 28,
 14,
 18,
 5,
 6,
 15,
 66,
 3,
 3,
 20,
 22

In [226]:
#Iterates through, combines the paragraphs into bigger ones of as close to 1500 characters as possible

combi_para = [para_lst[0]]
for para in para_lst[1:]:
    #print(combi_para[-1])
    #print(para)
    para_len = len(para)
    last_combi_len = len(combi_para[-1])
    #print("combi para len:", last_combi_len)
    #print("para len:", para_len)
    if para_len + last_combi_len <1500:
        combi_para[-1] += (" " + para)
        #print("new len:", len(combi_para[-1]))
    else:
        #print("too large")
        combi_para.append(para)

In [228]:
#these should be close, but the combi para will have a bit less than issue_txt because of the absence of all the new lines but more than para_lst because of the spaces added in concatenation

print(len(issue_txt))
print(sum([len(combi) for combi in combi_para]))
print(sum([len(para) for para in para_lst]))

66659
65947
65335


In [165]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)

In [229]:
# Display the text with the named entities in various colors
# You may need to first !pip install rich

# print header with colors
header_text = Text()
header_text.append("person", style="red bold")
header_text.append(" ")
header_text.append("location", style="green bold")
header_text.append(" ")
header_text.append("organization", style="blue bold")
rprint(header_text)

i = 0
for text in combi_para:
    i+=1
    print("Text", i)
    results = classifier(text)
    results_df = pd.DataFrame(results)

    # print source sentence with colors
    colored_text = Text()
    for word in text.split(" "):
        entity = None
        for result in results:
            #correcting for length 1 entities (like "S")
            if len(word) > 2 and len(result['word']) == 1:
                continue
            if word.startswith(result['word']):
                entity = result['entity']
                break
        if entity:
            # Use different colors for different entity types
            if entity == 'B-PER' or entity == 'I-PER':
                colored_text.append(word, style="red bold")
            elif entity == 'B-LOC' or entity == 'I-LOC':
                colored_text.append(word, style="green bold")
            elif entity == 'B-ORG' or entity == 'I-ORG':
                colored_text.append(word, style="blue bold")
            else:
                colored_text.append(word)
        else:
            colored_text.append(word)
        colored_text.append(" ")

    # print colored_text
    rprint(colored_text)

person location organization

Text 1


SG elects Fortier president; by-law amendment adopted New officer’s for the 1954-55 Student 
Government were the principal item of business for the first meeting of the new SG assembly 
last Tuesday. Al Fortier (ISL-col) was elected president of SG. Other officers elected were 
Emil John Writedns illegal, eight ballots void A major controversy in the re- Asked why he 
didn’t petition cent Student Government elec- and register as an official candison (ISL-col),
vice-president; Sabina Wagner (ISL-col), secretary; and Pat Parker (ISLbus), treasurer. An 
amendment to the SG by-laws was adopted which provides that any party receiving 25 per cent 
of the total vote in an SG election and holding a seat in the assembly is entitled to a 
committee-at-large, and thu S to a representative on the SG executive council. This gives the
Student Representative party a seat on the council, which it would not otherwise have had. 
The former by-law provided that any party with 20 per cent of the seats in SG was entitled to
a committee-at-large. Clark Kucheman (ISL-FTS) proposed an amendment whereby this requirement
could be waived by a two-thirds vote of the assembly Photo by Tipton Al Fortier, 
newly-elected Student Government president, discussing SG activities with members Jan Porter 
and Paul Bresiow. tions centered around the three seats in the Medical school. Although three
positions were open to Med school candidates, SRP ran none, and ISL had only two.

Text 2


After the ballots were counted it was reported that eight of the Med school ballots had been 
counted as void because each had the name of Fred Solomon written in. Solomon thereupon 
contested the ruling that the ballots were invalid and asked for the rule which governed 
write-ins. The rule governing write-ins, according to Jim Handler, chairman of the SG 
election and rules committee, is as follows: “Any writing on a ballot shall render the ballot
void.” This rule is interpreted by the E and R committee to cover any and all writing. 
Anderson wants UC support: ISL wins election tells why Douglas did not get here Douglas 
Anderson, Senator Douglas’ Illinois representative, denied categorically that he had ever 
spurned the support of the Independent date, Solomon replied: “For personal reasons I did not
know until late last week that I could spend any time in Student Government. “In this 
context,” he added, “writing means extraneous remarks, defamatory remarks after a candidate’s
name, and the like. It does not mean ‘no write-in votes allowed.’ I assumed a democratic 
election would allow a voter to vote for candidates of his choice, whether or not their names
appeared on the ballot.”

...

Soccer, Earlham, 9.30 a.m., Stagg Field Cross country, Wright and Wisconsin State, 12 noon, 
Washington Park Gordon’s Sport Shop Tennis Balls, $6.00 a dozen $16.50 Tennis Rockets, $9.95 
3757 Cottage Grove  *’ fusion V5ths Scotch l/5th Yankee 71 | What a mixture of fun! I Paul 
Douglas I Alex Mackenzie I A tasty dish of 4 pure whimsey Students presenting their ID cards 
at box office will be admitted for 50c any week night. Saturdays and Sundays until 5 p.m. Eye
Examinations Visual Training Dr. Kurt Rosenbaum optometrist 1132 E. 55th Street HYde Park 
3-8372 Louise Barker photographer "ivho captures your personality as well as your person'’ 
1457 E. 57th St. BU 8-0876 THE CHICAGO MAROON October 29, 1954 Page 8 Want Ads Lost am 
PHOTOGRAPHERS Randolph and Kar Analytic Geometry and Calculus book. No name inside. Finder 
please return to Edward Kowalski. Hitchcock Hall. Room 51. Services Moving? Call Prime 
Movers. Two husky students, car, trailer. Call MI 3-920 or PL 2-6412. Reasonable rates. 
Mathematical instruction to fit needs of individual or group. Soglin. ST 2-6727. Wanted Used 
lightweight woman’s bicycle. If you have one for sale call DO 3-1858 or RY 3-7915. Mother to 
care for my 14-months-old girl in her home; days; near University. PL 2-2958. For Sale Blue 
ribbed mohair couch and chair. Good condition. $37.50. NO 7-0615. 1947 Kaiser. Pretty fair 
condition. $85. Call MI 3-0800, Ext. 1588.

Text 47


Sofa. 17-inch Westinghouse TV. Upright vacuum cleaner. Bookcase. Philco radiophonograph. 
Desk. Best offer. BO 8-4036 before Sunday. Woodlawn near 53rd. 2-flat brick. 8 and 7. Each 
apartment. 4 bedrooms, Pi baths, living, dining, kitchen, screen porch. 8-room has many 
modern extras. 2-car brick garage, overhead door. Call owner weekends or after 4 p.m. FA 
4-7020. Quality photographic enlarger. Slightly used. Perfect condition. 40-60% off. At 
bookstore photographic. Joe Wolf, ES 5-1615. Drastically reduced—Revere tape radiorecorder. 
Slightly used. Excellent condition. Portable: 3 speed. Yours for balance payments due; $148. 
June Johnnaber. Ext. 3296. Evenings HY 3-2 23. s German monocular miscroscope. Five 
eye-pieces; four objectives. Wooden and plastic cases. $300. NO 7-3136. Help Wanted 4 
Salesmen. We pay salary plus commission. Good part-time job for students. Car not necessary. 
Call 7-8 p.m., CO 4-7600. Thompson Movers, 434 W. 125th Pl. Local moving express service to 
and from freight depots. Rates reasonable. MU 4-0435. John Sutcliffe. Carmen’s used furniture
store. Moving and light hauling. 1127 E. 55th St. MU 4-9003. Personal Farewell to all my past
ana present instructors and other friends. It has been both a pleasure and a wonderful 
experience knowing you all. I look forward to the day when we may meet again. Sincerely, Marv
Lyonus. CARMEN'S Used Furniture Store Typewriters and Desks Moving and Light Hauling 1127 E. 
55th MU 4-9003 SAVE TIME — SAVE MONEY

Text 48


N & R Bonded Movers — also — New and Used Furniture Mi 3-4196 1510 E. 55th The College 
LAUNDERETTE 1449 East 57th St. MU 4-9236 BETZ JEWELRY Unusual Jewelry Our Specialty Expert 
Jewelry and Watch Repair N.S.A. Discount to Students 1523 E. 53rd PL 2-3038 Choose an Expert 
THOMPSON MOVERS Local and Long Distance Moving CO 4-7600 Bibo, ergo sum Jimmy’s 1172 E. 55th 
MIDWAY 3-4433 1171 EAST 55th STREET ACASA Book Store Scholarly used books Typewriters bought,
sold and repaired 1117 E. 55lli St. Hl 3-9651 new and different stag or drag Dancing and 
Entertainment Tonight Music by Eddie Fox, His Orchestra and Vocalist from 9:00 P.M. Admission
$1.00 plus tax and that's all The Elms Hotel, East 53rd Street and Cornell Avenue We stock 
most blends of Twining Teas. Earl Grey, $1.75 Jasmine, $2.00. In half pound canisters. GET 
MUCH MORE FLAVOR MUCH LESS NICOTINE I Jisquhr. *WQ It's the FILTER that Counts and L*M has 
the Best! give you the Miracle Tip — the effec* tive filtration you need.. Cet much more 
flavor — much less nicotine — a light, mild smoke. Yes, — it s the filter that counts and L&M
has the best! BUY L&Ms King-size or regular. JUST WHAT THE DOCTOR ORDERED! Why do L&M sales 
soar higher every day? It’s the one filter cigarette that gives smokers a taste they can 
enjoy — a filter they can depend on. Now L&M comes king-size, too, at the same low price as 
regular. In either size —only L&M Filters 1 ^Yers ION

In [202]:
# Example 2 : Davlan/distilbert-base-multilingual-cased-ner-hrl
tokenizer = AutoTokenizer.from_pretrained("Davlan/distilbert-base-multilingual-cased-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/distilbert-base-multilingual-cased-ner-hrl")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)

In [203]:
i = 0
for text in [text1, text2, text3, text4, text5]:
    i+=1
    print("\n", "Text", i, "\n-------")
    results = classifier(text)
    results_df = pd.DataFrame(results)

    print(results_df)


 Text 1 
-------
   entity     score  index        word  start  end
0   B-LOC  0.788741     23           S    115  116
1   I-LOC  0.998884     24           .    116  117
2   I-LOC  0.999079     25       Ellis    118  123
3   I-LOC  0.998806     26      Avenue    124  130
4   B-PER  0.999920     30        Gary    145  149
5   I-PER  0.999754     31    Lawrence    150  158
6   B-ORG  0.999637     34     Chicago    166  173
7   I-ORG  0.999847     35        Fire    174  178
8   I-ORG  0.999868     36  Department    179  189
9   B-ORG  0.999890     79  University    400  410
10  I-ORG  0.999863     80    Security    411  419

 Text 2 
-------
  entity     score  index        word  start  end
0  B-LOC  0.996006      7         Ken     23   26
1  I-LOC  0.992652      8      ##wood     26   30
2  B-PER  0.999740     18         Tom     69   72
3  I-PER  0.999528     19       Smith     73   78
4  B-ORG  0.615618     24           N     93   94
5  I-LOC  0.669423     25  University     95  105
6 

In [204]:
# Display the text with the named entities in various colors
# You may need to first !pip install rich

# print header with colors
header_text = Text()
header_text.append("person", style="red bold")
header_text.append(" ")
header_text.append("location", style="green bold")
header_text.append(" ")
header_text.append("organization", style="blue bold")
rprint(header_text)

i = 0
for text in [text1, text2, text3, text4, text5]:
    i+=1
    print("Text", i)
    results = classifier(text)
    results_df = pd.DataFrame(results)

    # print source sentence with colors
    colored_text = Text()
    for word in text.split(" "):
        entity = None
        for result in results:
            #correcting for length 1 entities (like "S")
            if len(word) > 2 and len(result['word']) == 1:
                continue
            if word.startswith(result['word']):
                entity = result['entity']
                break
        if entity:
            # Use different colors for different entity types
            if entity == 'B-PER' or entity == 'I-PER':
                colored_text.append(word, style="red bold")
            elif entity == 'B-LOC' or entity == 'I-LOC':
                colored_text.append(word, style="green bold")
            elif entity == 'B-ORG' or entity == 'I-ORG':
                colored_text.append(word, style="blue bold")
            else:
                colored_text.append(word)
        else:
            colored_text.append(word)
        colored_text.append(" ")

    # print colored_text
    rprint(colored_text)

person location organization

Text 1


Six people were injured, three seriously, in a fire which occurred Monday morning in an apartment building at 5401 
S. Ellis Avenue. According to Gary Lawrence of the Chicago Fire Department, the fire was centered in the second and
third floors of the northeastern section of the building. It appeared that 7 units out of a total of 70 units in 
the building were destroyed. According to one member of University Security who was at the scene, the second and 
third floors were largely gutted and there was a possibility of water damage to the first floor.

Text 2


A man at the corner of Kenwood and 60th reported seeing a large dog. Tom Smith lives at 5527 N University. There is
a giant parade happening at 8191 Blackstone. There is a block party happening at Grant and Thompson.

...

Cardinal Bembo, I write to you from Timbuktu, scholarly capital of the world. I am not myself a scholar; I am a 
soldier and a statesman, currently in the process of wresting the throne of the Songhai Empire away from its 
undeserving current king, the miserable Sonni Baru. Soon I will move my permanent residence to the capital of Gao; 
but the scholars of the University of Sankore here in Timbuktu have begged me to write to you, specifically. So 
here we are.

Text 4


All Europe had known Milan would be the core of battle, armies poised so long, the strength of Europe concentrated 
where the road from Bernard's Pass (now Saint Bernard's Pass) touches Lombardy. The preparations were brave. Duke 
Ludovico, Cardinal Ascanio, and Count Galahad Sanseverino returned to the city, and there made great preparation 
for the grand visit of Emperor Maximilian, who came to be crowned with the Iron Crown of Lombardy.

Text 5


The book of the generation of Jesus Christ, the son of David, the son of Abraham. Abraham begat Isaac; and Isaac 
begat Jacob; and Jacob begat Judas and his brethren; and Judas begat Phares and Zara of Thamar;

In [205]:
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/roberta-large-ner-english")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/roberta-large-ner-english")
classifier = pipeline('ner', model=model, tokenizer=tokenizer)

In [206]:
i = 0
for text in [text1, text2, text3, text4, text5]:
    i+=1
    print("\n", "Text", i, "\n-------")
    results = classifier(text)
    results_df = pandas.DataFrame(results)

    print(results_df)


 Text 1 
-------
   entity     score  index         word  start  end
0     LOC  0.739124     23           ĠS    115  116
1     LOC  0.560388     24            .    116  117
2     LOC  0.863648     25       ĠEllis    118  123
3     LOC  0.596748     26      ĠAvenue    124  130
4     LOC  0.560389     27            .    130  131
5     PER  0.999706     30        ĠGary    145  149
6     PER  0.999638     31    ĠLawrence    150  158
7     ORG  0.998016     34     ĠChicago    166  173
8     ORG  0.999042     35        ĠFire    174  178
9     ORG  0.999490     36  ĠDepartment    179  189
10    LOC  0.560388     55            .    287  288
11    LOC  0.560389     73            .    371  372
12    ORG  0.998157     79  ĠUniversity    400  410
13    ORG  0.998840     80    ĠSecurity    411  419
14    LOC  0.560388    108            .    552  553

 Text 2 
-------
   entity     score  index         word  start  end
0     LOC  0.934733      7         ĠKen     23   26
1     LOC  0.942030      8  

In [207]:
# Display the text with the named entities in various colors
# You may need to first !pip install rich

# print header with colors
header_text = Text()
header_text.append("person", style="red bold")
header_text.append(" ")
header_text.append("location", style="green bold")
header_text.append(" ")
header_text.append("organization", style="blue bold")
rprint(header_text)

i = 0
for text in [text1, text2, text3, text4, text5]:
    i+=1
    print("\n", "Text", i, "\n-------")
    results = classifier(text)
    results_df = pd.DataFrame(results)

    # print source sentence with colors
    colored_text = Text()
    for word in text.split(" "):
        entity = None
        for result in results:
            result_word = result['word'].strip("Ġ")
            #correcting for length 1 entities (like "S")
            if len(word) > 2 and len(result_word) == 1:
                continue
            if len(word) >= 3 and len(result_word) <= 2:
                continue
            if word.startswith(result_word):
                entity = result['entity']
                break
        if entity:
            # Use different colors for different entity types
            if entity == 'PER':
                colored_text.append(word, style="red bold")
            elif entity == 'LOC':
                colored_text.append(word, style="green bold")
            elif entity == 'ORG':
                colored_text.append(word, style="blue bold")
            else:
                colored_text.append(word)
        else:
            colored_text.append(word)
        colored_text.append(" ")

    # print colored_text
    rprint(colored_text)

person location organization


 Text 1 
-------


Six people were injured, three seriously, in a fire which occurred Monday morning in an apartment building at 5401 
S. Ellis Avenue. According to Gary Lawrence of the Chicago Fire Department, the fire was centered in the second and
third floors of the northeastern section of the building. It appeared that 7 units out of a total of 70 units in 
the building were destroyed. According to one member of University Security who was at the scene, the second and 
third floors were largely gutted and there was a possibility of water damage to the first floor.


 Text 2 
-------


A man at the corner of Kenwood and 60th reported seeing a large dog. Tom Smith lives at 5527 N University. There is
a giant parade happening at 8191 Blackstone. There is a block party happening at Grant and Thompson.

...

Cardinal Bembo, I write to you from Timbuktu, scholarly capital of the world. I am not myself a scholar; I am a 
soldier and a statesman, currently in the process of wresting the throne of the Songhai Empire away from its 
undeserving current king, the miserable Sonni Baru. Soon I will move my permanent residence to the capital of Gao; 
but the scholars of the University of Sankore here in Timbuktu have begged me to write to you, specifically. So 
here we are.


 Text 4 
-------


All Europe had known Milan would be the core of battle, armies poised so long, the strength of Europe concentrated 
where the road from Bernard's Pass (now Saint Bernard's Pass) touches Lombardy. The preparations were brave. Duke 
Ludovico, Cardinal Ascanio, and Count Galahad Sanseverino returned to the city, and there made great preparation 
for the grand visit of Emperor Maximilian, who came to be crowned with the Iron Crown of Lombardy.


 Text 5 
-------


The book of the generation of Jesus Christ, the son of David, the son of Abraham. Abraham begat Isaac; and Isaac 
begat Jacob; and Jacob begat Judas and his brethren; and Judas begat Phares and Zara of Thamar;